In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 61 kB 438 kB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
     |████████████████████████████████| 895 kB 47.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import string
import re
import numpy as np
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import pandas as pd
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

import requests
import io
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def simplify(word):
	dump = ''
	temp = []
	listOfWords = list(filter(None,re.split("([A-Z][^A-Z]*)",word)))
	if len(listOfWords) == len(word):
		return word.lower()
	for i in range(len(listOfWords)):
		listOfWords[i] = listOfWords[i].lower()
		if len(listOfWords[i]) == 1:
			dump = dump + listOfWords[i]
			if dump in words.words() and len(dump) > 2:
				temp.append(dump)
				dump = ''
		else:
			temp.append(listOfWords[i])
	return temp

In [4]:
def get_vector(vec_type,max_squence_len,label_map=None):
    labels =[]
    text = []

    for i in range(len(lines)):
        temp = lines[i].split(' ',1)
        if len(temp)<2: continue
        labels.append(temp[0])
        text.append(temp[1].strip())
    map = get_glove_map("glove.twitter.27B.25d.txt")
    text_vec = []

In [5]:
def createTokens(data):
  listOfTweets = []
  listOfStances = []
  tweetVector = []
  
  for ind, row in data.iterrows():
    
    # Create a sentence using target and the tweet. Word vector will be formed from this.
    example_sentence = str(row["Target"]) + " " + str(row["text"])
    
    # Remove punctuation
    final_sentence = example_sentence.translate(string.punctuation)
    wordList = word_tokenize(final_sentence)
    wordList = [w.lower() for w in wordList]
    
    stop_words = set(stopwords.words('english'))
    wordList = [w for w in wordList if not w in stop_words] 
    
    finalList = []
    s = ' '.join([i for i in wordList if i.isalpha()])
    
    # create tokens from the string and stem them
    wordList = word_tokenize(s)
    
    for word in wordList:
      #to break any combined word into its components for eg, hashtags
      finalList += simplify(word)
    
    final_sentence = ' '.join(finalList)
    listOfTweets.append(word_tokenize(final_sentence))
  return listOfTweets

In [6]:
def labelStance(labelDict, data):
	for key, val in labelDict.items():
		data.loc[data["Stance"] == val, "Stance"] = int(key)
	return data

In [7]:
def getWordVector(word, model):
  wordVector = model[word]
  return wordVector

In [8]:
def createVector(data, model):
  data = createTokens(data)
  tweetVector = []
  vector = np.zeros_like(model["abortion"])
  for tweet in data:
    for word in tweet:
      vector = vector + getWordVector(word,model)
    tweetVector.append(vector)
  tweetVector = np.asarray(tweetVector)
  return tweetVector

In [9]:
def getStances(data):
  stanceList = []
  for ind, row in data.iterrows():
    stanceList.append(row["Stance"])
  return np.asarray(stanceList)

In [10]:
path ='https://raw.githubusercontent.com/tanvi-ap/Covid19StanceDetection/main/Train.csv'

r= requests.get(path)
dtf = pd.read_csv(io.StringIO(r.text),index_col=0)
dtf=dtf.rename(columns={"label":"Stance", "safe_text":"text"})
dtf["Target"]="Covid19 Vaccination"

In [33]:
dtf.head()

,text,Stance,agreement,Target
tweet_id,,,,
CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0,Covid19 Vaccination
E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0,Covid19 Vaccination
M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0,Covid19 Vaccination
1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0,Covid19 Vaccination
J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0,Covid19 Vaccination


In [11]:
X_train, X_test= train_test_split(dtf ,test_size=0.3)
X_train

,text,Stance,agreement,Target
tweet_id,,,,
HM920I8B,"""<user> Should parents be allowed to choose wh...",1.0,0.666667,Covid19 Vaccination
ZAJJ4EAN,<user> i bwt they wont because its a kid show....,0.0,1.000000,Covid19 Vaccination
4EOJAZJ2,Suppose I should find a measles vaccine.,1.0,0.666667,Covid19 Vaccination
FD7P3EYP,<user> <user> <user> that's what the whole #CD...,-1.0,1.000000,Covid19 Vaccination
IU7MOM0D,Free back-to-school immunizations offered arou...,1.0,0.666667,Covid19 Vaccination
...,...,...,...,...
45PYS286,The government flaw with these virus immunizat...,-1.0,1.000000,Covid19 Vaccination
OGF773B5,<user> I'll pull a Kristen bell and not let an...,1.0,0.666667,Covid19 Vaccination
07RW6IUA,<user> <user> I'm pissed. measles outbreak -l...,-1.0,0.333333,Covid19 Vaccination


In [12]:
classifiers = ['Support Vector Machine', 'Random Forest Classifier', 'Gradient Boosting Classifier', 'Logistic Regression', 'Neural Network', 'Gaussian NB', 'K Neighbors Classifier', 'Decision Tree Classifier']

# trainTweets = pd.read_csv('H:\Project Files\stance-detection\Dataset-SemEval2016/training.txt', sep='\t',header=0,encoding='utf-8')
# testTweets = pd.read_csv('H:\Project Files\stance-detection\Dataset-SemEval2016/test-gold.txt', sep='\t',header=0,encoding='utf-8')

labelDict = {0:"AGAINST", 1:"FAVOR", 2:"NONE"}
X_train = labelStance(labelDict, X_train)
X_test = labelStance(labelDict, X_test)   

print("Processing the dataset...........")
listOfTweets = pd.concat([X_train, X_test], axis=0)
listOfTweets = createTokens(listOfTweets)
uniqTrainTargets = X_train.Target.unique()

print("Processing the word2vec model...........")
model = Word2Vec(listOfTweets, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

Processing the dataset...........
Processing the word2vec model...........


In [13]:
totalAcc = []
Targetacc = []
for classifier in classifiers:
  print("\nThe Machine Learning Model Used For Classification : " + classifier)
  temp = []
  total = 0
  for target in uniqTrainTargets:
    trainTweetVectors = createVector(X_train[X_train["Target"]==target], model)
    testTweetVectors = createVector(X_test[X_test["Target"]==target], model)
    trainStances = getStances(X_train[X_train["Target"]==target])
    testStances = getStances(X_test[X_test["Target"]==target])


#     print("\nProcessing the input for targrt " + str(target) + ".....")
#     print("Training Sample size : " + str(trainTweetVectors.shape[0]))
#     print("Testing Sample size : " + str(testTweetVectors.shape[0]))

#    if classifier == 'Support Vector Machine':
 #     clf = SVC(kernel="rbf").fit(trainTweetVectors, trainStances)

    # elif classifier == 'Random Forest Classifier':
    #   clf = RandomForestClassifier(n_estimators=90).fit(trainTweetVectors, trainStances)

    # elif classifier == 'Gradient Boosting Classifier':
    #   clf = GradientBoostingClassifier().fit(trainTweetVectors, trainStances)

    # elif classifier == 'Logistic Regression':
    #   clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(trainTweetVectors, trainStances)

    # elif classifier == 'Neural Network':
    #   clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(trainTweetVectors, trainStances)

    # elif classifier == 'Gaussian NB':
    #   clf = GaussianNB().fit(trainTweetVectors, trainStances)

    # elif classifier == 'K Neighbors Classifier':
    #   clf = GaussianNB().fit(trainTweetVectors, trainStances)

    # elif classifier == 'Decision Tree Classifier':
    #   clf = tree.DecisionTreeClassifier().fit(trainTweetVectors, trainStances)

  #  acc = clf.score(testTweetVectors, testStances)
    # print("Test Accuracy is "+ str(round(acc*100,2)) +"%")
    # total = total + acc
    # temp.append(acc)
    
  # Targetacc.append(temp)
  # total = total/len(uniqTrainTargets)
  # print("\nTotal Test Accuracy is "+ str(round(total*100,2)) +"%")
  # totalAcc.append(total)


The Machine Learning Model Used For Classification : Support Vector Machine

The Machine Learning Model Used For Classification : Random Forest Classifier

The Machine Learning Model Used For Classification : Gradient Boosting Classifier


KeyboardInterrupt: ignored

In [14]:

X_Train,X_Test = train_test_split(dtf,test_size=0.3)


In [15]:
y_train = X_train["Stance"].values
y_test = X_test["Stance"].values

In [16]:
from keras.preprocessing.text import Tokenizer

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dtf['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15118 unique tokens.


In [17]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(dtf['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

max_len = 0
for t in X:
    if max_len < len(t):
        max_len = len(t)

X1 = tokenizer.texts_to_sequences(dtf['text'].values)
X1 = pad_sequences(X1, maxlen=max_len)

print('Shape of data tensor:', X.shape)

Shape of data tensor: (10001, 100)


In [18]:
Y = pd.get_dummies(dtf['Stance']).values

Y1 = pd.get_dummies(dtf['Stance']).values

print('Shape of label tensor:', Y.shape)

Shape of label tensor: (10001, 3)


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping

model = Sequential()
EMBEDDING_DIM = 100
MAX_NB_WORDS = 50000
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

print(model.summary())



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          5000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 5,080,703
Trainable params: 5,080,703
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
X = np.asarray(X_train)#.astype(np.float32)
Y = np.asarray(y_train).astype(np.float32)
X_test_ = np.asarray(X_test)
y_test_ = np.asarray(y_test).astype(np.float32) 
#history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [30]:
model.fit(X_train,y_train,validation_data=(X_test_,y_test_),epochs=120,batch_size=64)

ValueError: ignored

In [31]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dtf['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15118 unique tokens.


In [32]:
X = tokenizer.texts_to_sequences(dtf['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (10001, 250)


In [34]:
Y = pd.get_dummies(dtf['Stance']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (10001, 3)


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9000, 250) (9000, 3)
(1001, 250) (1001, 3)


In [40]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(13, activation='softmax'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

#history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [41]:
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
127/127 [==============================] - 106s 804ms/step - loss: 0.8201 - accuracy: 0.6248 - val_loss: 0.6923 - val_accuracy: 0.7200
Epoch 2/5
127/127 [==============================] - 102s 804ms/step - loss: 0.5613 - accuracy: 0.7741 - val_loss: 0.6504 - val_accuracy: 0.7433
Epoch 3/5
127/127 [==============================] - 101s 797ms/step - loss: 0.3887 - accuracy: 0.8543 - val_loss: 0.7323 - val_accuracy: 0.7244
Epoch 4/5
127/127 [==============================] - 101s 791ms/step - loss: 0.2750 - accuracy: 0.9037 - val_loss: 0.8912 - val_accuracy: 0.7100
Epoch 5/5
127/127 [==============================] - 101s 795ms/step - loss: 0.2007 - accuracy: 0.9278 - val_loss: 0.9400 - val_accuracy: 0.7111


In [42]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

32/32 [==============================] - 2s 75ms/step - loss: 0.9448 - accuracy: 0.6933
Test set
  Loss: 0.945
  Accuracy: 0.693
